# IDENTIFICACIÓN DE SISTEMAS

Profesor: Jairo Alberto Cuéllar Guarnizo  
Programa: Ingeniería en Automatización y Control

In [68]:
from sympy import MatrixSymbol, Matrix, Identity

import sympy as sym
import pandas as pd

sym.init_printing()
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import control
import random

# Métodos paramétricos de estimación

## Estimación Lineal por mínimos cuadráticos (LS) - Resumen

Para el siguiente modelo lineal:

$$y[k] = \phi[k]^T\theta + \varepsilon[k]$$

Inicialmente consideremos el error como 0, por tanto el modelo se reduce a $y[k] = \phi[k]^T\theta$. La idea de LLS es minimizar el valor de error entre las mediciones $y[k]$ y los valores estimados $\theta$ elevados al cuadrado, por tanto la función de costo a minimizar sería:

$$f(\theta) = \frac{1}{2}|y - \phi\theta|^2$$

En la sección anterior se llegó a la conclusión que el vector de parámetros estimados es equivalente a:

$$\theta = [\phi^T.\phi]^{-1}.\phi^T.y$$

$$\theta_{LS} = \phi^+.y$$

## Estimación Lineal por mínimos cuadráticos con pesos (WLS)

Considerando que la señal del error es de media cero, pero que contiene varianzas distintas, se podría evaluar la siguiente función de costos, que considera que ante cada medición se divide en la varianza de dicha medición. Por tanto la función de costo no tendría unidades:

$$f_{WLS}(\theta) = \sum_{k=1}^{N}\frac{(y[k]-\phi^T.\theta)^2}{\sigma_{\epsilon}[k]^2}$$

En este caso, se puede introducir una matriz de pesos, denominada W y que estaría dada por la siguiente relación:

$$W = \begin{bmatrix}\sigma_{\epsilon}[1]^{-2}& & \\ & ... & \\ & &\sigma_{\epsilon}[N]^{-2}\end{bmatrix}$$

Se podría reescribir la función de costo de forma vectorial así:

$$f_{WLS}(\theta) = |y-\phi.\theta|^2.W$$

$$f_{WLS}(\theta) = (y-\phi.\theta)^T.W.(y-\phi.\theta)$$

Calculando el gradiente en función de teta y despejando el estimador se obtiene la siguiente relación:

$$\theta_{WLS} = (\phi^T.W.\phi)^{-1}.\phi^T.W.y$$

Otra forma de obtener el vector de los parámetros estimados es introducir la matriz $W^{1/2}$, que es equivalente a la raiz cuadrada de la matriz de pesos W.

$$W^{1/2} = \begin{bmatrix}\sigma_{\epsilon}[1]^{-1}& & \\ & ... & \\ & &\sigma_{\epsilon}[N]^{-1}\end{bmatrix}$$

Tal que: $W = W^{1/2}.W^{1/2}$

Sustituyendo en la función de costo se tiene lo siguiente:

$$f_{WLS}(\theta) = |W^{1/2}.(y-\phi.\theta)|^2 = |W^{1/2}.y-W^{1/2}.\phi.\theta)|^2 $$

Se introducen dos nuevos vectores como se indica a continuación para el vector normalizados de regresión y para los valores medidos normalizados:

$$\tilde{y} = W^{1/2}.y$$

$$\tilde{\phi} = W^{1/2}.\phi$$

Aplicando los criterios de optimalidad a esta función de costo y despejando el estimador y teniendo en cuenta las 2 relaciones anteriores se podría indicar que el vector de parámetros estimados daría:

$$\theta_{WLS} = [\tilde{\phi}^T.\tilde{\phi}]^{-1}.\tilde{\phi}^T.\tilde{y}$$

$$\theta_{WLS} = \tilde{\phi}^+.\tilde{y}$$

Así que un problema de mínimos cuadrados con pesos, es solo un problema de mínimos cuadrados escalando la matriz de regresión y los valores medidos por la matriz $W^{1/2}$.

## Ejemplo 1:

Se espera poder determinar un modelo lineal dado por: $y = -6 + 2x$. Además, se tiene un vector de mediciones de longitud 5, dado por la siguiente relación:
$$y[k] = \begin{bmatrix}y[1]\\y[2]\\...\\y[N]\end{bmatrix}=\begin{bmatrix}-5.996\\-4.008\\-1.997\\-0.009 \\ 2.009\end{bmatrix}$$.

Considere a su vez que x está de 0 a 4.

In [101]:
#Valores de x = 0, 1, 2, 3, 4
x = np.linspace(0,4,5,dtype = "int")

#Valores medidos con un error y
y = np.array([-5.998,-4.005,-1.998,-0.008,2.002])

(array([0, 1, 2, 3, 4]), array([-5.998, -4.005, -1.998, -0.008,  2.002]))

La matriz de regresión, al ser un modelo lineal estaría dada por: $\phi = \begin{bmatrix}1 & x[1]\\1 & x[2]\\1 & x[3]\\1 & x[4]\\1 & x[5]\end{bmatrix}$. 

Y se debe considerar las covarianzas dadas por:

$W = \begin{bmatrix}0.063&0&0&0&0\\0&0.062&0&0&0\\0&0&0.25&0&0\\0&0&0&12345&0\\0&0&0&0&0.248\end{bmatrix}$

In [100]:
# La matriz de regresión considerando el modelo lineal
ones = np.ones(N)
phi = np.array([ones,x]).transpose()

# Varianzas para cada xk
var = np.array([0.063,0.062,0.251,15625,0.25])
_var = var**0.5

## Matriz de pesos (Covarianzas)
W = np.eye(N)*var
_W = np.eye(N)*_var
W,    _W

(array([[6.3000e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 6.2000e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.5100e-01, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.5625e+04, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.5000e-01]]),
 array([[  0.25099801,   0.        ,   0.        ,   0.        ,
           0.        ],
        [  0.        ,   0.24899799,   0.        ,   0.        ,
           0.        ],
        [  0.        ,   0.        ,   0.500999  ,   0.        ,
           0.        ],
        [  0.        ,   0.        ,   0.        , 125.        ,
           0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.5       ]]))

Se debe calcular los vectores normalizados 
$$\tilde{y} = W^{1/2}.y$$

$$\tilde{\phi} = W^{1/2}.\phi$$

In [102]:
## Vectores "y" y "phi" Normalizados usando W
_y = np.dot(_W,y)
_phi = np.dot(_W,phi)
_y,      _phi

(array([-1.50548605, -0.99723696, -1.00099601, -1.        ,  1.001     ]),
 array([[2.50998008e-01, 0.00000000e+00],
        [2.48997992e-01, 2.48997992e-01],
        [5.00999002e-01, 1.00199800e+00],
        [1.25000000e+02, 3.75000000e+02],
        [5.00000000e-01, 2.00000000e+00]]))

In [103]:
## Calculando los estimados THETA
phi_pr = np.dot(_phi.transpose(),_phi)
A = np.linalg.inv(phi_pr)
B = np.dot(_phi.transpose(),_y)

## Vector de parámetros estimados
theta = np.dot(A,B)
theta

array([-6.00282057,  1.99827364])